In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold, permutation_test_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

/Users/nataliele.zhang/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [37]:
train = pd.read_csv('train.csv').fillna(99)
train.head()

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
0,0,3,32,3.0,99.0,323011,3854,481,1975,1,...,99.0,99,99,99,99,99,99,99,99,99
1,1,2,26,99.0,8.0,268131,2441,344,1981,1,...,99.0,99,1,99,2,99,2,99,2,99
2,2,1,16,99.0,7.0,167581,754,143,1995,1,...,1.0,99,2,99,2,99,2,99,2,99
3,3,4,44,5.0,99.0,445071,5705,604,1980,1,...,99.0,99,2,99,2,99,99,99,99,99
4,4,4,43,99.0,6.0,436161,5645,592,1958,1,...,99.0,99,1,99,1,99,1,99,1,99


In [38]:
inc_col = ["DG3","DG4","DG6","DG12B_1","DG12B_2","DG12C_1","DG12C_2","DG13_1","DG13_2","DG13_3","DG14","DL0","DL1",
           "DL2","DL4_5","DL4_11","DL4_15","DL4_16","DL4_17","DL4_20","DL4_21","DL4_22","DL4_23","DL5","DL7","DL15",
           "DL26_5","DL27","DL28","G2P1_5","G2P1_6","G2P1_7","G2P1_11","G2P1_12","G2P1_15","G2P1_16","MT1A","MT2",
           "MT6","MT7","MT7A","MT9","MT18_2","MT18_3","MT18_4","MT18_5","FF1","FF2","FF14_20","FF16_12","FF16_13",
           "FF16_19","FF16_20","MM13","IFI16_1","IFI16_3","IFI24","FL1","FL3","FL4","FL6_1","FL6_2","FL7_6","FL8_3",
           "FL8_6","FL8_7","FL9A","FL9B","FL9C","FL10","FL12","FL13","FL14","FL16","FL17","FL18","FB19","FB20","FB21",
           "FB24","FB25","FB27_1","FB27_2","FB27_3","FB27_4","FB27_5","FB27_6","FB27_7","FB27_8","FB27_9","FB27_96",
           "LN1A","LN1B","LN2_1","LN2_2","LN2_3","LN2_4","GN1","GN2","GN3","GN4","GN5","RI4"]
x_cols = [x for x in train.columns if x in inc_col]

In [39]:
len(x_cols)

102

In [40]:
from catboost import CatBoostClassifier

In [41]:
X_train, X_test, y_train, y_test = train_test_split(train[x_cols], train['is_female'], test_size=0.33)

In [42]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [48]:
clf = CatBoostClassifier(eval_metric="AUC", one_hot_max_size=3, iterations=10)

In [49]:
clf.fit(X=X_train, y=y_train)

0:	learn: 0.9269084	total: 62.2ms	remaining: 560ms
1:	learn: 0.9400130	total: 120ms	remaining: 479ms
2:	learn: 0.9390820	total: 182ms	remaining: 425ms
3:	learn: 0.9419747	total: 245ms	remaining: 367ms
4:	learn: 0.9440616	total: 375ms	remaining: 375ms
5:	learn: 0.9469182	total: 460ms	remaining: 306ms
6:	learn: 0.9475105	total: 550ms	remaining: 236ms
7:	learn: 0.9462744	total: 746ms	remaining: 187ms
8:	learn: 0.9452277	total: 813ms	remaining: 90.3ms
9:	learn: 0.9460386	total: 879ms	remaining: 0us


In [50]:
y_pred_proba = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)

In [51]:
print confusion_matrix(y_test, y_pred)

[[2362  413]
 [ 394 2856]]


In [52]:
print classification_report(y_test, y_pred)

             precision    recall  f1-score   support

          0       0.86      0.85      0.85      2775
          1       0.87      0.88      0.88      3250

avg / total       0.87      0.87      0.87      6025



In [23]:
test = pd.read_csv('test.csv').fillna(-999)
test.head()

/Users/nataliele.zhang/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,13,48,50,55,90,125,144,162,163,164,166,168,169,170,171,172,174,176,241,243,245,247,251,254,273,289,291,292,293,295,296,312,313,314,331,343,365,373,375,396,413,439,490,619,623,630,633,638,642,644,669,712,759,768,788,809,828,928,953,978,1000,1001,1002,1003,1004,1023,1036,1040,1042,1063,1085,1098,1099,1120,1128,1135,1151,1152,1165,1167,1181,1192,1204,1205,1206,1207,1215,1225,1227,1229,1231,1233) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,test_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,DG3,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
0,0,4,41,-999.0,7.0,417211,4479,535,1979,8,...,2.0,-999,1,-999,3,-999,3,-999,3,-999
1,1,3,32,2.0,-999.0,322011,3803,476,1993,1,...,1.0,-999,1,-999,1,-999,1,-999,1,-999
2,2,3,36,5.0,-999.0,365011,5610,585,1980,3,...,2.0,-999,2,-999,2,-999,2,-999,2,-999
3,3,2,24,-999.0,7.0,247061,2550,350,1991,3,...,2.0,-999,2,-999,2,-999,2,-999,2,-999
4,4,3,35,-999.0,8.0,358071,3233,400,1985,3,...,1.0,-999,1,-999,1,-999,1,-999,1,-999


In [25]:
test_X = test[x_cols]

In [36]:
test_X.shape

(27285, 102)

In [30]:
predict_prob_test = clf.predict_proba(test_X)

In [31]:
test['is_female'] = pd.DataFrame(predict_prob_test)[1]

In [32]:
test = test[['test_id', 'is_female']]

In [33]:
test.head()

,test_id,is_female
0,0,0.511260
1,1,0.501127
2,2,0.511277
3,3,0.511260
4,4,0.511277


In [34]:
test.to_csv('result_20180207_b.csv')